# Synacor Challenge VM

**Solution Started**: July 7, 2025  
**Author**: Abbas Moosajee

This notebook loads and runs the Synacor challenge binary using a custom `VirtualMachine` implementation, highlights challenge codes, and displays the interactive game output in styled HTML.

In [ ]:
import os, re, time, copy, json
from collections import defaultdict, deque
from html import escape
from IPython.display import display, HTML
from VirtualMachine import VirtualMachine
start_time = time.time()

In [ ]:
# Load the binary program file
program_file = "challenge.bin"
file_path = os.path.join(os.getcwd(), program_file)  # assumes file in current working dir
vm_program = open(file_path, "rb").read()

In [ ]:
class SynacorConsole:
    track_backwards = {"north":"south", "south":"north", "east":"west","west":"east"}

    def __init__(self, software, visualize: bool = True):
        self.software = software
        self.visualize = visualize
        self.console = self.reset_console()

    def reset_console(self):
        console = VirtualMachine(self.software)
        self.game_map = defaultdict(list)
        self.overall_commands = []
        self.challenge_codes = {}
        self.valid_codes = set()
        self.code_no = 0
        return console

    @staticmethod
    def md5_hash(code: str) -> str:
        import hashlib
        return hashlib.md5(code.encode('utf-8')).hexdigest()

    def __extract_codes(self, text):
        """Extract mixed-case challenge codes and store them with their MD5 hashes."""
        codes = re.findall(r"\b(?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*\b", text)

        new_codes = []
        for code in codes:
            if code not in self.valid_codes:
                self.code_no += 1
                self.challenge_codes[self.code_no] = (code, self.md5_hash(code))
                new_codes.append(code)
                self.valid_codes.add(code)
        return new_codes

    def __format_terminal(self, text, code_color, input_color):
        """Highlight any detected code-like strings in the terminal."""
        lines = []
        for line in text.splitlines():
            # Simple input highlight: lines starting with '>'
            if line.strip().startswith('>>'):
                line = f"<span style='color:{input_color}'>{line}</span>"
            else:
                line = re.sub(      # Example: highlight code-like patterns
                    r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
                    rf"<span style='color:{code_color}'>\1</span>", line
                )
            lines.append(line)
        return "\n".join(lines)

    def display_terminal(self, text, actions):
        """Format and display terminal output with highlighted codes and interleaved actions."""
        self.color_dict = {
            "background": "#282828", "terminal": "#33FF00",
            "codes": "#FF0000", "input": "#0073FF"
        }

        # Normalize line endings and remove excessive empty lines
        text = re.sub(r'\n{3,}', '\n\n', text.strip())  # At most 2 newlines
        parts = re.split(r'(What do you do\?)', text)

        output_lines = []

        if actions:
            output_lines.append(f">> {actions[0]}")
        action_index = 1 if actions else 0

        # Now interleave remaining actions after each prompt
        i = 0
        while i < len(parts):
            output_lines.append(parts[i])
            input_prompt = "What do you do?"
            if i + 1 < len(parts) and parts[i + 1] == input_prompt:
                output_lines.append(input_prompt) # Append prompt
                if action_index < len(actions):   # Append next action if available
                    output_lines.append(f">> {actions[action_index]}")
                    action_index += 1
                i += 2
            else:
                i += 1

        combined_text = "\n".join(output_lines)
        longest_line = max(max(len(line) for line in combined_text.split('\n')), 150)

        html = (
            f"<div style='background-color: {self.color_dict['background']}; "
            f"width: {longest_line + 2}ch; padding: 0.75ex;'>"
            f"<pre style='background-color: {self.color_dict['background']}; "
            f"color: {self.color_dict['terminal']}; margin: 0; font-family: monospace; "
            f"width: {longest_line + 2}ch;'>"
            + self.__format_terminal(combined_text, self.color_dict['codes'], self.color_dict['input'])
            + "</pre></div>"
        )

        display(HTML(html))

    def play_game_manually(self, actions = []):
        """
        Game Instructions:
        - look: You may merely 'look' to examine the room, or you may 'look ' (such as 'look chair') to examine something specific.
        - go: You may 'go ' to travel in that direction (such as 'go west'), or you may merely '' (such as 'west').
        - inv: To see the contents of your inventory, merely 'inv'.
        - take: You may 'take ' (such as 'take large rock').
        - drop: To drop something in your inventory, you may 'drop '.
        - use: You may activate or otherwise apply an item with 'use '.
        """
        if actions:
            self.overall_commands.extend(actions)
        full_terminal = self.console.run_computer(actions)
        current_terminal = full_terminal[-1]
        self.display_terminal(current_terminal, actions)
        self.__extract_codes(current_terminal)
        return self.overall_commands

    def __parse_game_state(self, terminal):
        lines = terminal.splitlines()
        location, description, things, exits = ("", "", [], [])

        current_section = None
        for line_no, line in enumerate(lines):
            line = line.strip()
            if line.startswith("==") and line.endswith("=="):
                location = line.strip("= ").strip()
                description = lines[line_no + 1]
            elif line.startswith("Things of interest here:"):
                current_section = "things"
            elif line.startswith("There") and "exit" in line:
                current_section = "exits"
            elif line.startswith("-") and current_section:
                item = line[2:].strip()
                if current_section == "things":
                    things.append(item)
                elif current_section == "exits":
                    exits.append(item)
            else:
                current_section = None

        return location, description, things, exits

    def __generate_commands(self, current_state, visited, commands_list):
        location, desc, all_items, exits = self.__parse_game_state(current_state)
        print(self.md5_hash(desc))
        if all_items:
            for item in all_items:
                commands_list.append(f"take {item}")
                commands_list.append(f"use {item}")
        commands_list.append(f"{exits[0]}")
        self.game_map[location].extend(exits)
        # print(f"{self.past_loc}, {location}")
        self.past_loc = location
        # print(location, exits)
        visited.add((location, tuple(exits)))
        # commands_list.append("help")
        return commands_list, visited

    def nested_dict(self):
        return defaultdict(self.nested_dict)

    def add_path(self, map_dict, path):
        current = map_dict
        for step in path:
            current = current[step]

    def bfs(self):
        """Perform BFS to explore and map out the game world."""
        steps = 0
        BASE_ROOM = "Foothills"
        game_map = self.nested_dict()
        visited = set()
        room_info = {}
        inventory = []

        # Each queue entry: (console_state, actions_taken, path_taken)
        queue = [(self.console, [], [], [])]

        while queue and steps < 250:  # Limit steps for testing/debugging
            steps += 1
            state, input_actions, path, action_history = queue.pop(0)
            # print(input_actions)
            last_room = path[-1] if path else ""
            reverse_dir = self.track_backwards.get(last_room, "")

            # Prepare a clean game state and run the latest action
            game_copy = state.replicate()
            output = game_copy.run_computer(input_actions)
            terminal_output = output[-1]
            self.__extract_codes(terminal_output)
            actions = []
            # if len(self.challenge_codes) >= 4:
            #     if self.visualize:
            #         full_terminal = '\n\n'.join(output[1:])
            #         self.display_terminal(full_terminal, action_history)

            # Parse current state
            room, desc, items, exits = self.__parse_game_state(terminal_output)
            if room == "Ruins":
                print(room, desc)
            room_hash = self.md5_hash(desc)

            if (room_hash, tuple(inventory)) in visited:
                continue
            visited.add((room_hash, tuple(inventory)))
            room_info[room_hash] = (room, desc)

            # # If there's an item, pick it up
            if items:
                # print(f"Found items {items} {action_history}")
                inventory.append(items[0])
                item_action = f"take {items[0]}\nuse {items[0]}"
                actions.extend([item_action])
                action_history.extend([item_action])

            # Explore new directions
            for direction in exits:
                if direction == reverse_dir and items:
                    continue  # Don't go backwards immediately

                next_path = path + [direction]
                next_actions = actions + [direction]
                all_actions = action_history + [direction]
                self.add_path(game_map, next_path)
                queue.append((game_copy, next_actions, next_path, all_actions))

        # Optional debug output
        print("steps:", steps)
        # self.print_game_map(game_map)
        return

    def print_game_map(self, d=None, indent=2):
        """
        Pretty print the nested defaultdict tree as a standard dict.
        """
        if d is None:
            d = self.game_map  # or whatever your map variable is
        def to_dict(d):
            if isinstance(d, defaultdict):
                return {k: to_dict(v) for k, v in d.items()}
            return d
        print(json.dumps(to_dict(d), indent=indent))

    def play_game(self, actions = []):
        steps = 0
        self.past_loc = "Base"
        game_state = self.console.replicate()
        visited = set()

        while True:
            if actions:
                self.overall_commands.extend(actions)
            full_terminal = game_state.run_computer(actions)
            current_terminal = full_terminal[-1]
            self.__extract_codes(current_terminal)
            if self.visualize:
                self.display_terminal(current_terminal, actions)
            if steps >= 7:
                break
            actions, visited = self.__generate_commands(current_terminal, visited, [])
            steps += 1
        print(self.overall_commands)

        return self.overall_commands


In [ ]:
console = SynacorConsole(vm_program)
# command_list = console.play_game()
test = console.bfs()

In [ ]:
# console.play_game_manually()
# # game_actions = [
# # #     # "take tablet",
# # #     # 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'east',
# # #     # 'take empty lantern',
# # #     ]
# game_actions = [
#     'take tablet','doorway', 'north', 'north', 'bridge', 'continue', 'down',
#     'west', 'passage', 'ladder', 'north', 'north', 'north', 'ladder', 'cavern',
#     'use empty lantern', 'east', 'east'
#     ]
# all_actions = console.play_game_manually(game_actions)

In [ ]:
for code_no, code_group in console.challenge_codes.items():
    print(code_no, code_group)
print(f"Execution Time = {time.time() - start_time:.5f}s")